# ❇️ Esmeralda: Preceptoria Híbrida e IA Socrática
### O resgate da proficiência médica no Brasil

**Equipe:** Eduardo Chuairi (Produto), Marllon Pereira (IA), Phillipe Wolff (Back-end) e Jayme Ricardo (Front-end).

Este notebook documenta a arquitetura técnica e a prova de conceito (PoC) da Esmeralda. A aplicação utiliza o modelo **MedGemma 1.5 4B Multimodal** (HAI-DEF) acoplado a uma rigorosa Engenharia de Prompt para atuar como uma preceptora sênior. Em vez de fornecer diagnósticos prontos, o sistema utiliza o Método Socrático para forçar o raciocínio clínico de médicos em formação.

# Installs
Instalação e atualização das bibliotecas essenciais para a arquitetura do sistema, abrangendo orquestração RAG, vetorização de dados, inferência em GPU e estruturação do servidor de API.

**1. Orquestração e Ingestão de Dados (RAG):**
* `langchain`, `langchain-core`, `langchain-community`: Estrutura base para orquestrar o fluxo de dados entre a IA e os documentos externos.
* `langchain-text-splitters`: Segmenta os protocolos médicos extensos em blocos menores (chunks) para otimizar o processamento.
* `pypdf`: Motor de extração de texto para a leitura nativa dos protocolos em PDF.

**2. Vetorização e Busca Semântica:**
* `chromadb`: Banco de dados vetorial em memória, responsável por armazenar o conhecimento clínico do SUS.
* `sentence-transformers`, `langchain-huggingface`: Motores que convertem os textos médicos em vetores matemáticos (embeddings) para a busca de similaridade.

In [1]:
!pip install -U -q \
    langchain \
    langchain-community \
    langchain-core \
    langchain-huggingface \
    langchain-text-splitters \
    chromadb \
    sentence-transformers \
    transformers \
    accelerate \
    bitsandbytes \
    pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 11.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 38.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 68.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 25.6 MB/s eta 0:0

**3. Inferência e Otimização de GPU (Edge AI):**
* `torch`: Framework base de Deep Learning responsável por gerenciar os tensores na placa de vídeo.
* `transformers`, `huggingface_hub`: Gerenciam o download dos pesos e o pipeline de execução do modelo MedGemma 1.5.
* `accelerate`, `bitsandbytes`: Otimizam o uso da VRAM, viabilizando a execução local do modelo em hardware de consumo (Edge AI).
* `pillow`: Biblioteca de visão computacional para decodificar e processar imagens de exames médicos.

**4. Servidor de API RESTful:**
* `fastapi`, `uvicorn`: Estruturam o servidor assíncrono de alto desempenho que receberá as requisições do front-end.
* `requests`: Gerencia as requisições HTTP.
* `pyngrok`: Cria o túnel de rede que expõe temporariamente a API local para acesso externo.
* `nest-asyncio`: Modifica a gestão de eventos do ambiente interativo (Kaggle/Jupyter) para permitir a execução contínua do servidor assíncrono.

In [2]:
!pip install -q transformers pillow torch requests huggingface_hub accelerate fastapi uvicorn pyngrok nest-asyncio

# Login HF 
### Autenticação Segura
Autenticação segura na plataforma Hugging Face. O código utiliza o `UserSecretsClient` para resgatar o token de acesso (`HF_TOKEN`) configurado nas variáveis de ambiente restritas do Kaggle. Essa abordagem protege a credencial contra vazamentos no código-fonte, sendo uma etapa obrigatória para o download dos pesos do modelo MedGemma. O bloco inclui um tratamento de exceções (try/except) para alertar sobre falhas na leitura da chave.

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import os

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("✅ Login no Hugging Face realizado com sucesso!")
except Exception as e:
    print("⚠️ Não foi possível fazer login automático. Verifique se a Secret 'HF_TOKEN' está configurada.")
    # Se falhar, você pode descomentar a linha abaixo para logar manualmente:
    # login()

✅ Login no Hugging Face realizado com sucesso!


# RAG
### Base de Conhecimento (RAG)

Construção da base de conhecimento clínica do sistema utilizando Retrieval-Augmented Generation (RAG). O script executa o pipeline de ingestão e vetorização em quatro etapas fundamentais:

* **Ingestão de Dados (`PyPDFLoader`):** Busca dinamicamente os protocolos oficiais do SUS em formato PDF no diretório do Kaggle e extrai o conteúdo textual, ignorando falhas de leitura.
* **Segmentação (`TextSplitter`):** Divide os documentos extensos em fragmentos menores (*chunks* de 800 caracteres com sobreposição de 100), garantindo que a informação caiba na janela de contexto do LLM sem perda de continuidade.
* **Vetorização (`HuggingFaceEmbeddings`):** Utiliza um modelo *sentence-transformer* multilíngue otimizado para gerar representações semânticas densas (embeddings) dos trechos médicos.
* **Armazenamento e Recuperação (`Chroma`):** Inicializa um banco de dados vetorial em memória contendo as diretrizes vetorizadas. O sistema configura um *retriever* para resgatar os 3 fragmentos matematicamente mais relevantes a cada consulta do médico.

In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# --- CONFIGURAÇÃO DOS ARQUIVOS ---
target_files = [
    "Manejo_clinico_da_Dengue.pdf",
    "Manejo-dengue.pdf",
    "Manejo_clinico_da_Dengue.pdf",
    "dengue_aspecto_epidemiologicos_diagnostico_tratamento.pdf"
]

def find_file_path(filename, search_path='/kaggle'):
    """Procura o arquivo em todo o diretório do Kaggle"""
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# --- CARREGAMENTO ---
docs = []
print("--- 🔍 Buscando arquivos ---")

for file_name in target_files:
    full_path = find_file_path(file_name)
    if full_path:
        print(f"📄 Carregando: {file_name}")
        try:
            loader = PyPDFLoader(full_path)
            docs.extend(loader.load())
        except Exception as e:
            print(f"❌ Erro em {file_name}: {e}")
    else:
        print(f"⚠️ Arquivo não encontrado: {file_name}")

if not docs:
    raise ValueError("Nenhum documento carregado! Faça o upload dos PDFs no Kaggle.")

# --- DIVISÃO (SPLITTING) ---
# Chunks menores funcionam melhor para inserir no contexto do LLM
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
print(f"📚 Total de trechos processados: {len(splits)}")

# --- EMBEDDINGS & VECTOR STORE ---
print("🧠 Gerando banco vetorial (aguarde)...")
embedding_model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Cria o banco em memória
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Pega os 3 trechos mais relevantes

print("✅ Sistema RAG pronto para uso!")

2026-02-21 19:39:57.518586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771702797.762219      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771702797.834661      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771702798.419080      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771702798.419141      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771702798.419144      55 computation_placer.cc:177] computation placer alr

--- 🔍 Buscando arquivos ---
📄 Carregando: Manejo_clinico_da_Dengue.pdf
📄 Carregando: Manejo-dengue.pdf
📄 Carregando: Manejo_clinico_da_Dengue.pdf
📄 Carregando: dengue_aspecto_epidemiologicos_diagnostico_tratamento.pdf
📚 Total de trechos processados: 441
🧠 Gerando banco vetorial (aguarde)...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Sistema RAG pronto para uso!


# Pipe
### Instanciação do Modelo Multimodal

Carregamento do modelo fundacional e orquestração do Motor Socrático. Este bloco consolida o "cérebro" da aplicação e define as regras estritas de comportamento da IA através das seguintes etapas:

* **Instanciação do Modelo (`pipeline`):** Carrega os pesos do MedGemma 1.5 4B na memória da GPU. A utilização da precisão `bfloat16` otimiza severamente o consumo de VRAM, permitindo inferência local rápida (Edge AI).
* **Contingência Multimodal (`dummy_image`):** Implementa um *fallback* gerando uma matriz visual vazia (imagem preta). Isso satisfaz o requisito de entrada do modelo de visão-linguagem (VLM) para cenários onde o médico envia apenas texto, sem exames de imagem anexados.
* **Orquestração de Contexto (Retrieval):** A função `perguntar_ao_medgemma` aciona o *retriever* configurado no passo anterior para buscar os protocolos do SUS no banco vetorial que respondam à dúvida do usuário.
* **Engenharia de Prompt (Motor Socrático):** Constrói um prompt restritivo que injeta o contexto do RAG e força a IA a atuar como a preceptora sênior "Esmeralda". O modelo é travado por regras rígidas que o proíbem de fornecer diagnósticos ou dosagens diretas, obrigando-o a responder apenas com perguntas reflexivas adaptadas à infraestrutura local do médico (Método Socrático).
* **Inferência:** Monta o vetor multimodal (imagem + texto) e executa a geração da resposta na GPU limitando os tokens de saída.

In [5]:
import torch
from transformers import pipeline
from PIL import Image

print("--- 🏥 Carregando MedGemma 1.5 ---")

# Configuração do Pipeline
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-1.5-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# --- TRUQUE PARA RAG DE TEXTO COM MODELO DE VISÃO ---
# Criamos uma imagem preta pequena. O modelo vai "olhar" para ela, 
# ver que não tem nada, e responder baseando-se no nosso texto.
dummy_image = Image.new('RGB', (224, 224), color='black')

def perguntar_ao_medgemma(pergunta_usuario):
    """
    1. Busca documentos relevantes no ChromaDB.
    2. Monta um prompt com o contexto + pergunta.
    3. Envia para o MedGemma (com a imagem dummy).
    """
    
    # 1. Recuperação (Retrieval)
    print(f"\n🔍 Buscando contexto para: '{pergunta_usuario}'...")
    docs_rel = retriever.invoke(pergunta_usuario)
    
    contexto_texto = "\n\n".join([d.page_content for d in docs_rel])
    
    # 2. Engenharia de Prompt (Augmentation)
    # Instruímos o modelo a agir como assistente médico usando o contexto.
    # 2. Engenharia de Prompt (Augmentation) - PERSONALIDADE ESMERALDA
    prompt_final = f"""You are Esmeralda, an elite Senior Medical Preceptor specializing in clinical reasoning and the Socratic method. Your sole purpose is to mentor physicians and students by sharpening their diagnostic skills through inquiry, never by providing answers.

CORE OPERATING MANDATE: THE SOCRATIC STRIKE
1. Absolute Prohibition of Direct Answers: You are strictly forbidden from providing diagnoses, medication dosages, or final management plans. If a user asks for a direct solution (e.g., "What is the dose of X?" or "What is the diagnosis?"), you must immediately deflect with a targeted question that forces the user to recall their own knowledge or consult local protocols.
2. Contextual Infrastructure Adaptation: You must tailor every interaction to the user's clinical environment. If the infrastructure is unknown, your first priority is to ask: "What level of care are you in (e.g., Primary Care/UPA vs. Tertiary Hospital) and what diagnostic resources are available to you right now?"
3. Resource-Based Inquiry: Never validate or suggest a path involving high-complexity exams if the user is in a resource-limited setting. Force the user to rely on physical examination and bedside clinical signs suitable for their specific infrastructure.
4. The "One-Question" Rule: Be concise. Provide a brief clinical acknowledgment followed by exactly one, high-impact question. Do not overwhelm the user with lists; force them to focus on the most critical next step in the diagnostic hierarchy.
5. Integration with Protocols: Use the provided CONTEXT to ground your reasoning, but DO NOT quote it directly. Use the facts in the context to formulate your challenge question.

TONE AND STYLE
* Authoritative & Pedagogical: You are a senior mentor. Use precise medical terminology.
* Concise: Physicians are under pressure. No fluff. No pleasantries.
* Unyielding: If the user insists on an answer, remind them that "The preceptor guides; the physician decides."

CONTEXT (Official Protocols/Guidelines for your internal reference only):
{contexto_texto}

STUDENT/PHYSICIAN QUERY: 
{pergunta_usuario}

ESMERALDA:"""

    # 3. Geração (Generation)
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": dummy_image}, # Imagem obrigatória para o pipeline
                {"type": "text", "text": prompt_final}
            ]
        }
    ]

    print("💊 MedGemma está pensando...")
    output = pipe(text=messages, max_new_tokens=2000)
    resposta = output[0]["generated_text"][-1]["content"]
    
    return resposta, docs_rel

print("✅ MedGemma carregado e função de perguntas pronta!")



--- 🏥 Carregando MedGemma 1.5 ---


config.json:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cpu


✅ MedGemma carregado e função de perguntas pronta!


# SERVER
### API RESTful e Exposição do Servidor
Criação e exposição da API RESTful assíncrona que atua como ponte entre a interface do usuário (front-end) e o Motor Socrático. O código está estruturado nas seguintes etapas:

* **Configuração da API (`FastAPI` & `Pydantic`):** Inicializa o servidor e define o esquema de dados esperado (`MensagemUsuario`), que aceita o ID do usuário, o texto do caso clínico e, opcionalmente, uma imagem em formato Base64.
* **Processamento Multimodal (Decodificação e Fallback):** Intercepta o *payload* do front-end. Se uma imagem for enviada, o sistema limpa o cabeçalho e decodifica o Base64 para um objeto `PIL.Image`. Caso seja uma consulta puramente textual, o servidor gera automaticamente a matriz visual vazia (imagem preta) exigida pelo modelo de visão.
* **Integração e Inferência:** Monta a estrutura de mensagens exigida pelo MedGemma, encapsulando a imagem (ou o fallback) e o texto do usuário, e aciona o `pipe` instanciado no bloco anterior, limitando a resposta a 500 tokens.
* **Exposição de Rede (`Ngrok` & `Uvicorn`):** Limpa túneis residuais de execuções anteriores, autentica a chave do Ngrok via Kaggle Secrets e cria um túnel seguro. Isso expõe a porta 8000 do localhost do Kaggle para a internet pública, gerando o endpoint que será consumido pelo front-end. O `nest_asyncio` permite que o servidor `uvicorn` rode sem travar o *loop* de eventos do notebook.

In [6]:
import nest_asyncio
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import asyncio
from PIL import Image
import requests # Mantido caso precise no futuro, mas não usaremos mais para a imagem
from io import BytesIO
from typing import Optional
import base64
from kaggle_secrets import UserSecretsClient

# --- 1. CONFIGURAÇÃO DO SERVIDOR ---
nest_asyncio.apply()
app = FastAPI()
historico_conversas = {}

# Passo 1: Ajuste do modelo para 'imagem_url' correspondendo ao Frontend
class MensagemUsuario(BaseModel):
    user_id: str
    texto: str
    imagem_url: Optional[str] = None

# Health checker
@app.get("/")
def home():
    return {"status": "MedGemma API Online"}


@app.post("/chat")
async def responder_chat(dados: MensagemUsuario):
    uid = dados.user_id
    mensagem_usuario = dados.texto
    imagem_url = dados.imagem_url # Captura a variável corretamente

    if uid not in historico_conversas:
        historico_conversas[uid] = []

    historico_conversas[uid].append(f"Usuário: {mensagem_usuario}")

    resposta_ia = ""
    image = None

    try:
        # --- 2. PREPARAÇÃO DA IMAGEM ---
        if imagem_url:
            print("📥 Decodificando imagem enviada em Base64...")

            # O frontend manda no formato: "data:image/jpeg;base64,BASE64_STRING"
            # Precisamos dividir a string pela vírgula e pegar só a segunda parte (o código base64)
            if "," in imagem_url:
                base64_data = imagem_url.split(",")[1]
            else:
                base64_data = imagem_url # Caso já venha limpo sem o cabeçalho 'data:'

            # Decodifica o texto em Base64 transformando-o em Bytes
            image_bytes = base64.b64decode(base64_data)

            # Carrega a imagem a partir dos Bytes na memória usando PIL
            image = Image.open(BytesIO(image_bytes)).convert("RGB")

        else:
            # Modo apenas texto: Cria a imagem preta de placeholder para o modelo
            print("📝 Modo apenas texto (gerando imagem placeholder)...")
            image = Image.new('RGB', (224, 224), color='black')

        # --- 3. PREPARAÇÃO DO PROMPT E GERAÇÃO ---
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": mensagem_usuario}
                ]
            }
        ]

        # --- GERAÇÃO (Assumindo que 'pipe' está definido e configurado no seu ambiente) ---
        output = pipe(text=messages, max_new_tokens=500)
        resposta_ia = output[0]["generated_text"][-1]["content"]

    except Exception as e:
        print(f"❌ Erro: {e}")
        resposta_ia = f"Desculpe, ocorreu um erro técnico: {str(e)}"

    historico_conversas[uid].append(f"MedGemma: {resposta_ia}")

    return {
        "resposta": resposta_ia,
        "historico": historico_conversas[uid]
    }

# --- 4. EXPOSIÇÃO VIA NGROK ---
# IMPORTANTE: Garanta que 'userdata' e 'pipe' estão declarados antes no seu código/notebook
user_secrets = UserSecretsClient()
NGROK_TOKEN = user_secrets.get_secret("NGROK_TOKEN")
ngrok.set_auth_token(NGROK_TOKEN)

# Limpa túneis antigos
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)
ngrok.kill()

public_url = ngrok.connect(8000)
print(f"🌍 Endpoint Público: {public_url}")

config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()

🌍 Endpoint Público: NgrokTunnel: "https://uncapitalistic-gibson-germane.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [55]
